In this notebook, the practice of the lifetime package and estimate customer lifetime value is conducted.

Reference: 
1. clv learning: https://www.youtube.com/watch?v=486x8ccQThE
2. dataset used: https://www.kaggle.com/datasets/nathaniel/uci-online-retail-ii-data-set
3. notebook looked as helping hand: https://www.kaggle.com/code/halimedogan/crm-analytics-cltv/notebook

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import summary_data_from_transaction_data as summary
from lifetimes.utils import _customer_lifetime_value 

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")
sns.set_palette("pastel")

In [2]:
import os
print(os.getcwd())

c:\Users\Monika\Projects\Customer Life-time Value\Learning\clv_practice1_uci


In [3]:
df1 = pd.read_excel(r"uci_online_retail_II.xlsx", sheet_name="Year 2009-2010")
df2 = pd.read_excel(r"uci_online_retail_II.xlsx", sheet_name="Year 2010-2011")

In [4]:
df1.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [5]:
df2.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
def check_df(dataframe):
    print("################ Shape ####################")
    print(dataframe.shape)
    print("############### Types #####################")
    print(dataframe.dtypes)
    print("############### Unique Value Counts ###################")
    print(dataframe.nunique())
    print("############### NA Counts ########################")
    print(dataframe.isnull().sum())

In [7]:
check_df(df1)

################ Shape ####################
(525461, 8)
############### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
############### Unique Value Counts ###################
Invoice        28816
StockCode       4632
Description     4681
Quantity         825
InvoiceDate    25296
Price           1606
Customer ID     4383
Country           40
dtype: int64
############### NA Counts ########################
Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64


In [8]:
check_df(df2)

################ Shape ####################
(541910, 8)
############### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
############### Unique Value Counts ###################
Invoice        25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
Price           1630
Customer ID     4372
Country           38
dtype: int64
############### NA Counts ########################
Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64


In [ ]:
df1.groupby('Invoice').first()

,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
Invoice,,,,,,,
489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom
489436,48173C,DOOR MAT BLACK FLOCK,10,2009-12-01 09:06:00,5.95,13078.0,United Kingdom
489437,22143,CHRISTMAS CRAFT HEART DECORATIONS,6,2009-12-01 09:08:00,2.10,15362.0,United Kingdom
489438,21329,DINOSAURS WRITING SET,28,2009-12-01 09:24:00,0.98,18102.0,United Kingdom
...,...,...,...,...,...,...,...
C538121,22461,SAVOY ART DECO CLOCK,-1,2010-12-09 15:36:00,12.75,15535.0,United Kingdom
C538122,22444,GROW YOUR OWN PLANT IN A CAN,-1,2010-12-09 15:38:00,1.25,14696.0,United Kingdom
C538123,22331,WOODLAND PARTY BAG + STICKER SET,-1,2010-12-09 15:41:00,1.65,12605.0,Germany


In [24]:
df1_nullCID_Invoice = df1.loc[df1['Customer ID'].isna()]['Invoice'].unique()
df1_withCID_Invoice_record = df1.loc[~df1['Customer ID'].isna()]['Invoice']

df2_nullCID_Invoice = df2.loc[df2['Customer ID'].isna()]['Invoice'].unique()
df2_withCID_Invoice_record = df2.loc[~df2['Customer ID'].isna()]['Invoice']

print('Count of invoices with a Customer ID also appear without a Customer ID')
print('df1:', 
        df1_withCID_Invoice_record.isin(df1_nullCID_Invoice).sum())

print('df2:',
        df2_withCID_Invoice_record.isin(df2_nullCID_Invoice).sum())

Count of invoices with a Customer ID also appear without a Customer ID
df1: 0
df2: 0


Given the above, we can conclude that the null Customer IDs cannot be imputed and used for analysis.

In [ ]:
df1.groupby('Invoice')[['Customer ID']].nunique().eq(1).all(axis=1).sum()

np.int64(23587)

In [ ]:
def transform_data(data):
    data = data.copy()
    data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
    data['Amount'] = data['Quantity'] * data['Price']
    data.groupby('InvoiceDate').agg(custom_funct)
    return data

In [ ]:
(df2['Quantity'] == 0).sum

np.int64(0)